In [ ]:
import os
import pymysql

# 測試連線
conn = pymysql.connect(
    host=os.getenv('DB_HOST', 'db'),
    user=os.getenv('DB_USER', 'admin'),
    password=os.getenv('DB_PASSWORD', 'password'),
    database='project_db'
)
print("連線成功！")


連線成功！


In [1]:
# --- M1 初始化單元格 ---
# 可以將這層隔出來成為測試是否正常運作的py檔
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy
import os

# 1. 建立引擎
engine = create_engine("mysql+pymysql://admin:password@db:3306/project_db")

# 2. 檢查套件是否成功永久安裝
print(f"套件驗證成功！SQLAlchemy 版本: {sqlalchemy.__version__}")

# 3. 如果資料表是空的，自動匯入 (下次進來只要跑這格就好)
csv_path = "cleaned_data_revise_2.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.to_sql('temp_raw_data', con=engine, if_exists='replace', index=False)
    print(f"資料已自動更新，目前總筆數: {len(df)}")

套件驗證成功！SQLAlchemy 版本: 2.0.45


In [2]:
# 交接給哲廣
# 讓這裡可以連接到container外的csv檔案
import pandas as pd
from sqlalchemy import create_engine, text
import os

# 1. 重新定義 engine (確保變數存在)
DB_URL = "mysql+pymysql://admin:password@db:3306/project_db"
engine = create_engine(DB_URL)

# 2. 設定 CSV 路徑 (既然你放進 backend 了，直接寫檔名即可)
csv_path = "/app/import_data/cleaned_data_revise_2.csv"

try:
    # 3. 讀取與匯入
    df_to_import = pd.read_csv(csv_path)
    
    with engine.connect() as conn:
        # 將資料寫入資料庫
        df_to_import.to_sql('temp_raw_data', con=engine, if_exists='replace', index=False)
        print(f"成功！已匯入 {len(df_to_import)} 筆資料到 temp_raw_data 表中。")
        
        # 4. 立即測試查詢
        df_check = pd.read_sql(text("SELECT * FROM temp_raw_data LIMIT 5"), conn)
        print("資料庫前五筆資料如下：")
        display(df_check)
        
except FileNotFoundError:
    print(f"找不到檔案：{csv_path}。請確認檔案是否已移動到 backend 資料夾下。")
except Exception as e:
    print(f"匯入失敗：{e}")

成功！已匯入 17880 筆資料到 temp_raw_data 表中。
資料庫前五筆資料如下：


,title,telecommuting,has_company_logo,has_questions,employment_type,fraudulent,in_balanced_dataset,country,industry_group,edu_level
0,Marketing Intern,0,1,0,Unknown,0,0,US,Unknown,Unknown
1,Customer Service - Cloud Video Production,0,1,0,Full-time,0,0,NZ,Corporate_Services,Unknown
2,Commissioning Machinery Assistant (CMA),0,1,0,Unknown,0,0,US,Unknown,Unknown
3,Account Executive - Washington DC,0,1,0,Full-time,0,0,US,Tech,Bachelor
4,Bill Review Manager,0,1,1,Full-time,0,0,US,Health_Science,Bachelor


In [5]:
from sqlalchemy import create_engine, text
import pandas as pd
import os

# 1. 建立引擎
db_user = os.getenv('DB_USER', 'admin')
db_password = os.getenv('DB_PASSWORD', 'password')
db_name = os.getenv('DB_NAME', 'project_db')
DB_URL = f"mysql+pymysql://{db_user}:{db_password}@db:3306/{db_name}"
engine = create_engine(DB_URL)

# 2. 實作聚合查詢 (根據你的欄位修改：統計各產業的職缺數與遠端比例)
def get_industry_stats():
    # 改用 industry_group 作為分類，telecommuting 作為聚合指標
    sql_agg = text("""
    SELECT 
        industry_group, 
        COUNT(*) as job_count, 
        SUM(telecommuting) as remote_count,
        AVG(telecommuting) * 100 as remote_percentage
    FROM temp_raw_data 
    GROUP BY industry_group
    ORDER BY job_count DESC
    """)
    with engine.connect() as connection:
        return pd.read_sql(sql_agg, connection)

# 3. 實作分頁 (Pagination)
def get_paginated_jobs(page=1, size=10):
    offset = (page - 1) * size
    sql_page = text("SELECT * FROM temp_raw_data LIMIT :size OFFSET :offset")
    with engine.connect() as connection:
        return pd.read_sql(sql_page, connection, params={"size": size, "offset": offset})

# --- 執行測試 ---
try:
    print("正在執行【產業別】聚合查詢...")
    df_summary = get_industry_stats()
    print("查詢成功！")
    display(df_summary.head(10)) # 顯示前 10 筆

    print("\n正在執行分頁查詢 (第 1 頁)...")
    df_page = get_paginated_jobs(page=1, size=5)
    display(df_page)

except Exception as e:
    print(f"執行失敗: {e}")

正在執行【產業別】聚合查詢...
查詢成功！


,industry_group,job_count,remote_count,remote_percentage
0,Unknown,5995,243.0,4.0534
1,Tech,4375,317.0,7.2457
2,Corporate_Services,1554,51.0,3.2819
3,Education,1062,10.0,0.9416
4,Finance,1030,47.0,4.5631
5,Consumer_Retail,1000,24.0,2.4000
6,Health_Science,863,20.0,2.3175
7,Industrial_Energy,784,17.0,2.1684
8,Media_Entertainment,585,21.0,3.5897
9,Logistics_Travel,449,9.0,2.0045



正在執行分頁查詢 (第 1 頁)...


,title,telecommuting,has_company_logo,has_questions,employment_type,fraudulent,in_balanced_dataset,country,industry_group,edu_level
0,Marketing Intern,0,1,0,Unknown,0,0,US,Unknown,Unknown
1,Customer Service - Cloud Video Production,0,1,0,Full-time,0,0,NZ,Corporate_Services,Unknown
2,Commissioning Machinery Assistant (CMA),0,1,0,Unknown,0,0,US,Unknown,Unknown
3,Account Executive - Washington DC,0,1,0,Full-time,0,0,US,Tech,Bachelor
4,Bill Review Manager,0,1,1,Full-time,0,0,US,Health_Science,Bachelor
